### Year 0 Topology Analysis

In [1]:
import pandas as pd
import altair as alt

##### Area Totals - Year 0 Topology 

In [2]:
data_tot = pd.read_csv('savnw_tot.csv')
data_tot

,Area,Number,From Generation,To Load,To(+)/From(-) Ties,To Losses
0,FLAPCO,1,1288,1021,251,15
1,LIGHTCO,2,1199,1277,-98,21
2,WORLD,5,276,425,-152,4


##### Total generation and Load

In [3]:
total_gen = data_tot['From Generation'].sum()
total_gen

2763

In [4]:
total_load = data_tot['To Load'].sum()
total_load 

2723

##### Breakdown of Generation in each areas

In [5]:
data_gen = pd.read_csv('gen_sav.csv', header=1, skiprows=0)
data_gen.to_csv('data_gen_0.csv',sep='&',index= False)

##### Breakdown of Load in each areas 

In [6]:
data_load = pd.read_excel('rls.xlsx')
data_load[['Load','ID','Pload-Y0R','Qload-Y0R']]

,Load,ID,Pload-Y0R,Qload-Y0R
0,153,1,170.29728,85.1904
1,154,1,510.97536,383.3568
2,154,2,340.59456,298.1664
3,203,1,255.40416,127.7856
4,205,1,1021.86720,596.3328
5,3005,1,85.10688,42.5952
6,3007,1,170.29728,63.8928
7,3008,1,170.29728,63.8928


##### Installed Capacity

In [7]:
total_capacity = data_gen['Pmax'].sum()
total_capacity

4153.25

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [8]:
data_load = pd.read_csv('loading_savnw.csv', header=1, skiprows=0)

In [9]:
data_fil = data_load
data_load.columns

Index(['From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA'],
      dtype='object')

In [10]:
data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')

##### Branch and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [11]:
transformer_loading = data_fil[data_fil['From Bus V']!=data_fil['To Bus V']].reset_index(drop=True)
branch_loading = data_fil[data_fil['From Bus V'] == data_fil['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [12]:
data = transformer_loading[['From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-12-19dfca162667>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [13]:
data_l = branch_loading[['From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-13-647cbf347804>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram is plotted to check the no of transformer branches in the above defined loading bracket 

In [14]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 200, width = 400)

alt.Chart(...)

It can be seen that most of the branches falls into a bracket of loading between 50% to 70% 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [15]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 200, width = 400)

alt.Chart(...)

It can be seen that most of the branches falls into a bracket of loading between 25% to 50% 

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [16]:
transformer_overload = transformer_loading[transformer_loading['%MVA'] > 80.0]
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_transformer = transformer_overload['Branch'].unique()
print(list_transformer)

['3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1']


<ipython-input-16-3463096cce2d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \


In [17]:
transformer_overload

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
21,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4765,90.0,80.0,120.42,0.03,11.03,150.0,75.94,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...


##### Checking line branch loading greater than 80% 

In [18]:
branch_loading = data_fil[data_fil['From Bus V'] == data_fil['To Bus V']].reset_index(drop=True)
branch_loading[branch_loading['%I'] > 80.0]

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V


##### Bus Voltage

In [19]:
data_volt = pd.read_csv('savnw_volt.csv', header=1, skiprows=0)

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [20]:
data_v = data_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [21]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 120, width = 400)

alt.Chart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [22]:
data_fil = data_volt[data_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)
data_fil

,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,201,HYDRO,2,500.0,1.058,528.829,1.05<Vbus<1.1
1,3018,CATDOG_G,5,13.8,1.057,14.589,1.05<Vbus<1.1


##### Voltage Lower Limits

In [23]:
data_volt[data_volt['Bus Voltage(PU)']<0.95]

,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range


#### Conclusion 

Year 0 Topology Analysis was carried out for the reference load scenario.It was seen that
1. No overloading was reported for any line branches
2. The transformer branch reporting an overload > 80% is
    * 3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1
3. The buses violating the upper voltage limit are
    * Buses 201, 308